In [31]:
from mip import Model, CBC ,MAXIMIZE,CONTINUOUS, BINARY, xsum, OptimizationStatus

In [4]:
class Problem:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
        self.objective_coeffs = objective_coeffs #coef das funcoes objetivo.
        self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
        self.constraints_rhs = constraints_rhs # lados direitos das restricoes

    #def __str__(self): #funcao para printar o problema
    #   return f"Maximize {self.objective_coeffs} \nSubject to {self.constraints_coeffs} <= {self.constraints_rhs}"

In [12]:
class No:
    def __init__(self, model, solucao=None, no_pai=None, branching_var=None, branch_value=None):
        self.no_pai = no_pai
        self.model = model
        self.branching_var = branching_var
        self.branch_value = branch_value
        self.solucao = solucao
    
    ''' def __str__(self):
            for var in self.model.vars:
                print(f"{var.name} = {var.x}")
            return self.model.objective_value'''

In [18]:
def ler_dados(nome_arquivo):
    with open(nome_arquivo, 'r') as arq:
        linhas = arq.readlines()

    num_variaveis, num_restricoes = map(int, linhas[0].split())

    # coeficientes da funcao objetivo
    obj_coef = list(map(float, linhas[1].split()))

    restricoes = [] # coeficientes das restricoes
    rhs = [] # right hand side das restricoes

    for i in range(2, num_restricoes+2):
        restricoes.append(list(map(float, linhas[i].split()))[:-1])
    
    for i in range(2, num_restricoes+2):
        #rhs.append(list(map(float,linhas[i].split()[-2:-1])))
        rhs.append(float(linhas[i].split()[-1]))

    return Problem(obj_coef, restricoes, rhs)

In [23]:
main_path = '/Users/dorie/Downloads/UFPB/P4/PO/github/PESQUISA_OPERACIONAL/testes_utilizados'
arq1 = main_path + '/teste1.txt'
arq2 = main_path + '/teste2.txt'
arq3 = main_path + '/teste3.txt'
arq4 = main_path + '/teste4.txt'
#C:\Users\dorie\Downloads\UFPB\P4\PO\github\Pesquisa_Operacional\testes_utilizados\teste1.txt

In [32]:
def modelo(problema):


    qtd_restricoes = len(problema.constraints_coeffs)
    qtd_variaveis = len(problema.objective_coeffs)



    model = Model(sense=MAXIMIZE, solver_name=CBC)
    x = [model.add_var(var_type=CONTINUOUS,name=f"x_{i}",lb=0,ub=1.0) for i in range(len(problema.constraints_coeffs))]


    '''print("Valores das variáveis x antes da otimização:")
    for variavel in x:
        print(f"{variavel.name}: {variavel.x}")'''

    model.objective = xsum(problema.objective_coeffs[i]*x[i] for i in range(qtd_variaveis)) 
    
    for i in range(qtd_restricoes):
        exp_restricao = xsum(problema.constraints_coeffs[i][j]*x[j] for j in range(qtd_variaveis))
        model += exp_restricao <= problema.constraints_rhs[i]
    
    

    
    '''print("Status da otimização:", status)
    print("Valor da função objetivo:", model.objective_value)'''

    lista_solucao = []
    #print("Solution:")
    for v in model.vars:
        lista_solucao.append(v.x)

    return model

In [28]:
problem_arq1  = ler_dados(arq1)
print(len(problem_arq1.constraints_coeffs))

11


In [33]:
problem1 = modelo(problem_arq1)
root = No(problem1)

print(root.model.objective_value)

queue = [root]
nofinal = queue.pop(0)
print(nofinal.model.objective_value)

for var in nofinal.model.vars:
    print(f"{var.name} = {var.x}")

print(nofinal.model.status)

28.749999999999996
28.749999999999996
x_0 = 0.0
x_1 = 0.8749999999999997
x_2 = 0.0
x_3 = 0.0
x_4 = 1.0
x_5 = 1.0
x_6 = 0.0
x_7 = 0.0
x_8 = 0.0
x_9 = 0.0
x_10 = 0.0
OptimizationStatus.OPTIMAL


In [ ]:
queue = [1,2,3]
queue.pop(0)
queue[0]

2

In [36]:
# identificar branching variable

def identify_branching_variable(node):
    for var in node.model.vars:
        if var.x != 0.0 and var.x != 1.0:
            return var
    return None



In [35]:
# funcao bound

class BranchAndBound():

    def __init__(self):
        self.primal = 0
        self.Z = 0
        self.solution_vars = []

    def bound(self, node):
        node.model.optimize()

        if node.model.status == OptimizationStatus.INFEASIBLE:
            return "INVIAVEL",[], None
        
        elif node.model.status == OptimizationStatus.NO_SOLUTION_FOUND:
            return "NO SOLUTION",[], None

        elif node.model.objective_value <= self.primal: #podado/cortado/limitante
            return "PRUNING",[], None

        elif identify_branching_variable(node) == None:
            print('Solução integral encontrada')
            return "INTEGRALIDADE",node.model.vars,node.model.objective_value

        return "FRACIONARIO",node.model.vars,node.model.objective_value


    def branch_and_bound(problema_arq,self,node):
        model_problem = modelo(problema_arq)
        root = No(model_problem)
        queue = [root]

        while queue:
            status, solution, objective = self.bound(queue[0])

            if status in ['INVIAVEL','NO SOLUTION','PRUNING']:
                queue.pop(0)

            #elif status = 'INTEGRALIDADE':


            
            elif status == 'FRACIONARIO':
                branch_var = identify_branching_variable(node)
                no_explorado = queue.pop(0)

                # xj = 0
                novo_no1 = no_explorado.copy()
                novo_no1.model = no_explorado.model.copy()
                novo_no1.model += branch_var == 0

                # xj = 1
                novo_no2 = no_explorado.copy()
                novo_no2.model = no_explorado.model.copy()
                novo_no2.model += branch_var == 1

                queue.append(novo_no1)
                queue.append(novo_no2)




    pass





In [39]:
queue = [1,2,34]
a = queue.pop(0)
print(queue)
print(a)

[2, 34]
1


In [53]:
class ponto:
    def __init__(self,num,palavra):
        self.num = num
        self.palavra = palavra

    def __copy__(self):
        return (self.num, self.palavra)

p1 = ponto(12,'a')
p2 = p1
p2.num = p1.num + 1


a
